In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [2]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

# 결측치가 자동으로 채워짐
for i in train.columns[train.dtypes == "object"]: 
    train[i] = label.fit_transform(list(train[i]))
    
for i in test.columns[test.dtypes == "object"]:
    test[i] = label.fit_transform(list(test[i]))

In [3]:
train1 = train.drop(['등록차량수','단지코드'], axis=1)
test1 = test.drop(['단지코드'], axis=1)
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   총세대수                          2952 non-null   int64  
 1   임대건물구분                        2952 non-null   int64  
 2   지역                            2952 non-null   int64  
 3   공급유형                          2952 non-null   int64  
 4   전용면적                          2952 non-null   float64
 5   전용면적별세대수                      2952 non-null   int64  
 6   공가수                           2952 non-null   float64
 7   자격유형                          2952 non-null   int64  
 8   임대보증금                         2952 non-null   int64  
 9   임대료                           2952 non-null   int64  
 10  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 11  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 12  단지내주차면수                       2952 non-null   float64
dtypes: 

In [6]:
pred = 0
best_score = 0

k_fold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for train_idx, test_idx in k_fold.split(train1):
    x_train, x_test = train1.iloc[train_idx], train1.iloc[test_idx]
    
    # Catboost 사용할 때는 종속변수를 scaling하는게 좋다.
    y_train, y_test = np.log(train['등록차량수']).iloc[train_idx], np.log(train['등록차량수']).iloc[test_idx]
    
    cat = CatBoostRegressor(verbose=100,
                            eval_metric = 'MAE',
                            iterations = 200, 
                            learning_rate = 0.01)
    
    cat.fit(x_train, y_train,
            eval_set=(x_test, y_test),
            early_stopping_rounds=100,
            cat_features = [1,2,3,7])
    
    pred += cat.predict(test1) / 5 # 평균
    
    best_score += cat.best_score_["validation"]["MAE"] / 5

0:	learn: 0.7119650	test: 0.7449833	best: 0.7449833 (0)	total: 25.4ms	remaining: 5.05s
100:	learn: 0.3707454	test: 0.3967963	best: 0.3967963 (100)	total: 2.91s	remaining: 2.85s
199:	learn: 0.2550683	test: 0.2752871	best: 0.2752871 (199)	total: 5.65s	remaining: 0us

bestTest = 0.2752871154
bestIteration = 199

0:	learn: 0.7256256	test: 0.6909547	best: 0.6909547 (0)	total: 23.9ms	remaining: 4.76s
100:	learn: 0.3771297	test: 0.3628776	best: 0.3628776 (100)	total: 2.9s	remaining: 2.84s
199:	learn: 0.2574032	test: 0.2528919	best: 0.2528919 (199)	total: 5.66s	remaining: 0us

bestTest = 0.2528918996
bestIteration = 199

0:	learn: 0.7177133	test: 0.7348304	best: 0.7348304 (0)	total: 23.8ms	remaining: 4.73s
100:	learn: 0.3712152	test: 0.3853340	best: 0.3853340 (100)	total: 2.87s	remaining: 2.81s
199:	learn: 0.2539865	test: 0.2660152	best: 0.2660152 (199)	total: 5.66s	remaining: 0us

bestTest = 0.2660151782
bestIteration = 199

0:	learn: 0.7186543	test: 0.7123978	best: 0.7123978 (0)	total: 23.4m

In [7]:
best_score

0.26053827832966864

In [8]:
np.exp(pred)

array([556.7540551 , 654.94484083, 563.93851908, ..., 289.64933233,
       293.45847104, 294.80962846])